#                         COMP 448/548 – Medical Image Analysis
#                                                Homework #2
## Batuhan Yalçın
### 64274
### May 10, 2022

In [ ]:
# Homework 07: Linear Discriminant Analysisn
## Batuhan Yalçın
### May 6, 2022import numpy as np
import math
from PIL import Image
from numpy import asarray
from skimage.color import rgb2gray
from sklearn import svm
from numpy import ma
import copy
import os
import cv2
import glob
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from statsmodels.stats.contingency_tables import mcnemar

# Part I

## calculateCooccurrenceMatrix Function

In [ ]:
def calculateCooccurrenceMatrix(grayImg, binNumber, di, dj):
    bin_image = copy.deepcopy(grayImg)
    
    for i in range(len(grayImg)):
        for j in range(len(grayImg[0])):
            pixel_bin = int(math.floor(grayImg[i][j])/(len(grayImg[0])/binNumber))
            bin_image[i][j] = pixel_bin
    
    M = [[0 for i in range(binNumber)] for j in range(binNumber)]
    for i in range(binNumber):
        for j in range(binNumber):
            for k in range(len(grayImg)):
                for l in range(len(grayImg[0])):
                    if (k - di > -1) and (k - di < len(grayImg)) and (l - dj > -1) and (l - dj < len(grayImg)) :
                        if (bin_image[k][l] == i) and (bin_image[k-di][l - dj] == j):
                            M[i][j]+=1

    return M
    

## calculateAccumulatedCooccurrenceMatrix Function

In [ ]:
def calculateAccumulatedCooccurrenceMatrix(grayImg, binNumber, d):
    accM = np.zeros((binNumber,binNumber))
    accM += np.array(calculateCooccurrenceMatrix(grayImg, binNumber, d, 0))
    accM += np.array(calculateCooccurrenceMatrix(grayImg, binNumber, d, d))
    accM += np.array(calculateCooccurrenceMatrix(grayImg, binNumber, 0, d))  
    accM += np.array(calculateCooccurrenceMatrix(grayImg, binNumber, -d, d))
    accM += np.array(calculateCooccurrenceMatrix(grayImg, binNumber, -d, 0))
    accM += np.array(calculateCooccurrenceMatrix(grayImg, binNumber, -d, -d))
    accM += np.array(calculateCooccurrenceMatrix(grayImg, binNumber, 0, -d))
    accM += np.array(calculateCooccurrenceMatrix(grayImg, binNumber, d, -d))
    return accM

## calculateCooccurrenceFeatures Function

In [ ]:
def calculateCooccurrenceFeatures(accM):
    normalized_acc_m = accM / np.sum(accM)
    
    # The angular second moment
    ang_nd_moment = np.sum(np.power(normalized_acc_m, 2))
    
    # Maximum probability
    max_prob = np.max(normalized_acc_m)
    
    N = normalized_acc_m.shape[0]
    i = np.arange(1, N+1)
    j = np.arange(1, N+1)
    i.shape = (N, 1)
    j.shape = (1, N)
    
    # Inverse difference moment
    inverse_difference_moment = np.sum(normalized_acc_m / (1 + np.power(i - j, 2)))
    
    # Contrast
    contrast = np.sum(normalized_acc_m * np.power(i - j, 2))
    
    # Entropy 
    entropy = -np.sum(normalized_acc_m * (ma.log(normalized_acc_m).filled(0)))
    
    ii = np.arange(1, N+1)
    jj = np.arange(1, N+1)
    sum_i = np.sum(normalized_acc_m, 1)
    sum_j = np.sum(normalized_acc_m, 0)
    mu_x = np.sum(sum_i * i)
    mu_y = np.sum(sum_j * j)
    standart_d_x = np.sqrt(np.sum(sum_i * np.power(i - mu_x, 2)))
    standart_d_y = np.sqrt(np.sum(sum_j * np.power(j - mu_y, 2)))
    to_sum = np.sum((i * j) * normalized_acc_m)
    # correlation 
    correlation = (to_sum - mu_x * mu_y) / (standart_d_x * standart_d_y)
    return [ang_nd_moment, max_prob, inverse_difference_moment, contrast, entropy, correlation]

# Part II

## Accumulated co-occurrence matrix  Extracting the six texture features

## Linear svm

## Note that below part is take so long so go below and direclty upload the datas from features which I already get features data and store in cvs

In [35]:
## Note that below part is take so long so go below and direclty upload the datas from features which I already get features data and store in cvs# get all images and their features for training
bin_number = 8
d = 10
training_dir = "dataset//training//"
features = []
for img_file in glob.glob(training_dir + "*.jpg"):
    print(img_file)
    img = cv.imread(img_file, 0)
    acc_m = calculateAccumulatedCooccurrenceMatrix(img, bin_number, d)
    features.append(calculateCooccurrenceFeatures(acc_m))
train_features=np.array(features)    
training_labels = np.loadtxt("dataset//training_labels.txt")
training_labels = training_labels.tolist()
# save to csv file
np.savetxt('train_features.csv', features, delimiter=',')

dataset//training\tr1.jpg
dataset//training\tr10.jpg
dataset//training\tr100.jpg
dataset//training\tr101.jpg
dataset//training\tr102.jpg
dataset//training\tr103.jpg
dataset//training\tr104.jpg
dataset//training\tr105.jpg
dataset//training\tr106.jpg
dataset//training\tr107.jpg
dataset//training\tr108.jpg
dataset//training\tr109.jpg
dataset//training\tr11.jpg
dataset//training\tr110.jpg
dataset//training\tr111.jpg
dataset//training\tr112.jpg
dataset//training\tr113.jpg
dataset//training\tr114.jpg
dataset//training\tr115.jpg
dataset//training\tr116.jpg
dataset//training\tr117.jpg
dataset//training\tr118.jpg
dataset//training\tr119.jpg
dataset//training\tr12.jpg
dataset//training\tr120.jpg
dataset//training\tr121.jpg
dataset//training\tr122.jpg
dataset//training\tr123.jpg
dataset//training\tr124.jpg
dataset//training\tr125.jpg
dataset//training\tr126.jpg
dataset//training\tr127.jpg
dataset//training\tr128.jpg
dataset//training\tr129.jpg
dataset//training\tr13.jpg
dataset//training\tr130.jp

In [ ]:
training_labels = np.loadtxt("dataset//training_labels.txt")
print(training_labels)
training_labels = training_labels.tolist()
print(training_labels)
# load from the csv file
train_features_part2 = np.loadtxt('train_features_part2.csv',delimiter=',')

## Note that below part is take so long so go below and direclty upload the datas from features which I already get features data and store in cvs

In [36]:
## Note that below part is take so long so go below and direclty upload the datas from features which I already get features data and store in cvstest_dir ="dataset//test//"
test_features = []
for img_file in glob.glob(test_dir + "*.jpg"):
    img = cv.imread(img_file, 0)
    acc_m = calculateAccumulatedCooccurrenceMatrix(img, bin_number, d)
    test_features.append(calculateCooccurrenceFeatures(acc_m))
test_labels = np.loadtxt("dataset//test_labels.txt")
test_labels = test_labels.tolist()
# save to csv file
np.savetxt('test_features3.csv', features, delimiter=',')

In [270]:
test_labels = np.loadtxt("dataset//test_labels.txt")
test_labels = test_labels.tolist()
# load from the csv file
test_features = np.loadtxt('test_features_part2.csv',delimiter=',')

## Linear svm

In [335]:
## Accumulated co-occurrence matrix  Extracting the six texture features
C_values = [0.1, 1, 5, 10, 50, 100, 250, 500, 1000, 5000]
Gamma_values = np.linspace(0.1,10,100)
scaler = StandardScaler()
train_features_fitted = scaler.fit_transform(train_features)
train_features_fitted = np.array(train_features_fitted)
test_features_fitted = scaler.fit_transform(test_features)
test_features_fitted = np.array(test_features_fitted)


for c in C_values:
    svc = SVC(kernel='linear', class_weight='balanced', C=c)
    model = svc.fit(features_fitted, training_labels)
    training_pred = np.array(model.predict(features_fitted))
    test_pred = np.array(model.predict(test_features_fitted))

    print(c)
    print('Overall Accuracy for train  ', accuracy_score(training_labels, training_pred))
    print('Class Accuracy for train ', confusion_matrix(training_labels, training_pred, normalize="true").diagonal())
    print('Overall Accuracy : ', accuracy_score(test_labels, test_pred))
    print('Class Accuracy : ', confusion_matrix(test_labels, test_pred, normalize="true").diagonal())

for c in (np.linspace(0.1,6,60)):
    svc = SVC(kernel='linear', class_weight='balanced', C=c)
    model = svc.fit(train_features_fitted, training_labels)
    training_pred_lin = np.array(model.predict(features_fitted))
    test_pred_lin = np.array(model.predict(test_features_fitted))
    np.savetxt('test_pred_lin', test_pred_lin, delimiter=',')

    print(c)
    print('Overall Accuracy for train  ', accuracy_score(training_labels, training_pred_lin))
    print('Class Accuracy for train ', confusion_matrix(training_labels, training_pred_lin, normalize="true").diagonal())
    print('Overall Accuracy : ', accuracy_score(test_labels, test_pred_lin))
    print('Class Accuracy : ', confusion_matrix(test_labels, test_pred_lin, normalize="true").diagonal())
#Best train accuracy
c=5000
#Best test accuracy while protecting the train accuracy
c=5.2
C_values.append(5.2)
svc = SVC(kernel='linear', class_weight='balanced', C=c)
model = svc.fit(train_features_fitted, training_labels)
training_pred_lin = np.array(model.predict(train_features_fitted))
test_pred_lin = np.array(model.predict(test_features_fitted))
np.savetxt('test_pred_lin', test_pred_lin, delimiter=',')


0.1
Overall Accuracy for train   0.6666666666666666
Class Accuracy for train  [0.36666667 0.82954545 0.76315789]
Overall Accuracy :  0.4375
Class Accuracy :  [0.27083333 0.45614035 0.61538462]
1
Overall Accuracy for train   0.6612903225806451
Class Accuracy for train  [0.36666667 0.81818182 0.76315789]
Overall Accuracy :  0.4444444444444444
Class Accuracy :  [0.33333333 0.42105263 0.61538462]
5
Overall Accuracy for train   0.6612903225806451
Class Accuracy for train  [0.36666667 0.78409091 0.84210526]
Overall Accuracy :  0.4097222222222222
Class Accuracy :  [0.25       0.40350877 0.61538462]
10
Overall Accuracy for train   0.6505376344086021
Class Accuracy for train  [0.38333333 0.76136364 0.81578947]
Overall Accuracy :  0.4236111111111111
Class Accuracy :  [0.29166667 0.38596491 0.64102564]
50
Overall Accuracy for train   0.6612903225806451
Class Accuracy for train  [0.38333333 0.77272727 0.84210526]
Overall Accuracy :  0.4166666666666667
Class Accuracy :  [0.27083333 0.38596491 0.641

In [417]:
for c in (np.linspace(0.1,0.2,10)):
    if (c==0.16666666666666669):
        c=5000
    if (c==0.17777777777777778):
        c=5.2
    svc = SVC(kernel='linear', class_weight='balanced', C=c)
    model = svc.fit(train_features_fitted, training_labels)
    training_pred_lin = np.array(model.predict(features_fitted))
    test_pred_lin = np.array(model.predict(test_features_fitted))
    np.savetxt('test_pred_lin', test_pred_lin, delimiter=',')
    if(accuracy_score(test_labels, test_pred_lin)>0.46 or c==5.2 or c==5000):
        print("its okey!!")
        print(c)
        print('Overall Accuracy for train  ', accuracy_score(training_labels, training_pred_lin))
        print('Class Accuracy for train ', confusion_matrix(training_labels, training_pred_lin, normalize="true").diagonal())
        print('Overall Accuracy : ', accuracy_score(test_labels, test_pred_lin))
        print('Class Accuracy : ', confusion_matrix(test_labels, test_pred_lin, normalize="true").diagonal())
#Best train accuracy
c=5000
#Best test accuracy 
c=0.196
C_values.append(0.196)
#Best test accuracy while protecting the train accuracy
c=5.2
svc = SVC(kernel='linear', class_weight='balanced', C=c)
model = svc.fit(train_features_fitted, training_labels)
training_pred_lin = np.array(model.predict(train_features_fitted))
test_pred_lin = np.array(model.predict(test_features_fitted))
np.savetxt('test_pred_lin', test_pred_lin, delimiter=',')


its okey!!
5000
Overall Accuracy for train   0.6666666666666666
Class Accuracy for train  [0.4        0.77272727 0.84210526]
Overall Accuracy :  0.4097222222222222
Class Accuracy :  [0.25       0.38596491 0.64102564]
its okey!!
5.2
Overall Accuracy for train   0.6612903225806451
Class Accuracy for train  [0.36666667 0.78409091 0.84210526]
Overall Accuracy :  0.4166666666666667
Class Accuracy :  [0.25       0.40350877 0.64102564]
its okey!!
0.18888888888888888
Overall Accuracy for train   0.6505376344086021
Class Accuracy for train  [0.3        0.82954545 0.78947368]
Overall Accuracy :  0.4722222222222222
Class Accuracy :  [0.27083333 0.52631579 0.64102564]
its okey!!
0.2
Overall Accuracy for train   0.6559139784946236
Class Accuracy for train  [0.3        0.82954545 0.81578947]
Overall Accuracy :  0.4652777777777778
Class Accuracy :  [0.27083333 0.50877193 0.64102564]


In [337]:
# from list C=5.2 chosen for better acurracy rbf with gamma looked
for c in C_values:
    for gamma in Gamma_values:
        svc = SVC(kernel='rbf', class_weight='balanced', C=c, gamma=gamma)
        model = svc.fit(train_features_fitted, training_labels)
        training_pred = np.array(model.predict(train_features_fitted))
        test_pred = np.array(model.predict(test_features_fitted))
        
        #print(c,gamma)
        if(accuracy_score(test_labels, test_pred)>0.47):
            print(c,gamma)
            print("its okey")
            print('Overall Accuracy for train  ', accuracy_score(training_labels, training_pred))
            print('Class Accuracy for train ', confusion_matrix(training_labels, training_pred, normalize="true").diagonal())
            print('Overall Accuracy : ', accuracy_score(test_labels, test_pred))
            print('Class Accuracy : ', confusion_matrix(test_labels, test_pred, normalize="true").diagonal())
        
        

        
#Best Train and test accuracy
c_ref=50
gamma=3.4

svc = SVC(kernel='rbf', class_weight='balanced', C=c, gamma=gamma)
model = svc.fit(train_features_fitted, training_labels)
training_pred_rbf = np.array(model.predict(train_features_fitted))
test_pred_rbf = np.array(model.predict(test_features_fitted))
np.savetxt('test_pred_rbf', test_pred_rbf, delimiter=',')

10 0.1
its okey
Overall Accuracy for train   0.6881720430107527
Class Accuracy for train  [0.35       0.84090909 0.86842105]
Overall Accuracy :  0.4722222222222222
Class Accuracy :  [0.27083333 0.56140351 0.58974359]
50 3.4000000000000004
its okey
Overall Accuracy for train   1.0
Class Accuracy for train  [1. 1. 1.]
Overall Accuracy :  0.4722222222222222
Class Accuracy :  [0.29166667 0.66666667 0.41025641]
50 3.5000000000000004
its okey
Overall Accuracy for train   1.0
Class Accuracy for train  [1. 1. 1.]
Overall Accuracy :  0.4722222222222222
Class Accuracy :  [0.29166667 0.66666667 0.41025641]
100 3.4000000000000004
its okey
Overall Accuracy for train   1.0
Class Accuracy for train  [1. 1. 1.]
Overall Accuracy :  0.4722222222222222
Class Accuracy :  [0.29166667 0.66666667 0.41025641]
100 3.5000000000000004
its okey
Overall Accuracy for train   1.0
Class Accuracy for train  [1. 1. 1.]
Overall Accuracy :  0.4722222222222222
Class Accuracy :  [0.29166667 0.66666667 0.41025641]
250 3.400

In [329]:
def contingency_table(truth, comporison1, comporison2, val=0):
    table = [[0,0],[0,0]]
    tf_comporison1 = truth == comporison1
    tf_comporison2 = truth == comporison2
    if val != 0:
        tf_comporison1 = tf_comporison1[truth == val]
        tf_comporison2 = tf_comporison2[truth == val]
    table[0][0] = np.sum(np.logical_and(tf_comporison1, tf_comporison2))
    table[0][1] = np.sum(np.logical_and(tf_comporison1, np.logical_not(tf_comporison2)))
    table[1][0] = np.sum(np.logical_and(np.logical_not(tf_comporison1), tf_comporison2))
    table[1][1] = np.sum(np.logical_and(np.logical_not(tf_comporison1), np.logical_not(tf_comporison2)))
    return table





training_pred_lin = np.array(training_pred_lin)
training_pred_rbf = np.array(training_pred_rbf)
test_pred_lin = np.array(test_pred_lin)
test_pred_rbf = np.array(test_pred_rbf)
training_labels = np.array(training_labels)
test_labels = np.array(test_labels)
for i in range(4):
    table_train = contingency_table(training_labels, training_pred_lin, training_pred_rbf, i)
    result_train = mcnemar(table_train, exact=False, correction=True)
    print("Result_Train\n",result_train)
    
    table_test = contingency_table(test_labels, test_pred_lin, test_pred_rbf, i)
    result_test = mcnemar(table_test, exact=False, correction=True)
    print("Result_Test\n",result_test)

Result_Train
 pvalue      0.23013934044341314
statistic   1.44
Result_Test
 pvalue      0.2385928293164321
statistic   1.3888888888888888
Result_Train
 pvalue      1.0
statistic   0.0
Result_Test
 pvalue      0.22067136191984324
statistic   1.5
Result_Train
 pvalue      0.34278171114790873
statistic   0.9
Result_Test
 pvalue      1.0
statistic   0.0
Result_Train
 pvalue      0.22067136191984324
statistic   1.5
Result_Test
 pvalue      0.24821307898992026
statistic   1.3333333333333333


In [330]:
training_labels

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 3., 3., 3., 3., 3.,
       3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.,
       3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.])

# Part III

In [338]:
# get all images and their features for training
bin_number = 8
d = 10
N = 4


## Note that below part is take so long so go below and direclty upload the datas from features which I already get features data and store in cvs

In [303]:
# Part IIITrain_features_part3 = []
for img_file in glob.glob(training_dir + "*.jpg"):
    img = cv.imread(img_file, 0)
    x_dimension, y_dimension = img.shape
    height = x_dimension // N
    width = y_dimension // N
    acc_m_list = []
    for i in range(N):
        for j in range(N):
            patch = img[i*height:(i+1)*height, j*width:(j+1)*width]
            acc_m = calculateAccumulatedCooccurrenceMatrix(patch, bin_number, d)
            acc_m_list.append(calculateCooccurrenceFeatures(acc_m))


    Train_features_part3.append(np.mean((np.array(acc_m_list)), 0))
    
Train_features_part3 = np.array(Train_features_part3)
np.savetxt('Train_features_part3.csv', Train_features_part3, delimiter=',')

## Note that below part is take so long so go below and direclty upload the datas from features which I already get features data and store in cvs

In [177]:
Test_features_part3 = []
for img_file in glob.glob(test_dir + "*.jpg"):
    img = cv.imread(img_file, 0)
    x_dimension, y_dimension = img.shape
    height = x_dimension // N
    width = y_dimension // N
    acc_m_list = []
    for i in range(N):
        for j in range(N):
            patch = img[i*height:(i+1)*height, j*width:(j+1)*width]
            acc_m = calculateAccumulatedCooccurrenceMatrix(patch, bin_number, d)
            acc_m_list.append(calculateCooccurrenceFeatures(acc_m))


    Test_features_part3.append(np.mean((np.array(acc_m_list)), 0))
    
Test_features_part3 = np.array(Test_features_part3)
np.savetxt('Test_features_part3.csv', Test_features_part3, delimiter=',')

## Train Features

In [339]:
training_labels = np.loadtxt("dataset//training_labels.txt")
training_labels = training_labels.tolist()
training_labels = np.array(training_labels)## Note that below part is take so long so go below and direclty upload the datas from features which I already get features data and store in txt
# load from the csv file
train_features_part3 = np.loadtxt('train_features_part3.csv',delimiter=',')

## Test Features

In [340]:
test_labels = np.loadtxt("dataset//test_labels.txt")
test_labels = test_labels.tolist()
test_labels = np.array(test_labels)
# load from the csv file
test_features_part3 = np.loadtxt('test_features_part3.csv',delimiter=',')


In [306]:
#I just debug small error
print(len(training_labels))
len(test_features_part3)


186


144

## Found best C for linear kernel

In [341]:

features_fitted_part3 = scaler.fit_transform(train_features_part3)
test_features_fitted_part3 = scaler.fit_transform(test_features_part3)
features_fitted_part3 = np.array(features_fitted_part3)
test_features_fitted_part3 = np.array(test_features_fitted_part3)
for c in C_values:
    svc = SVC(kernel='linear', class_weight='balanced', C=c)
    model = svc.fit(features_fitted_part3, training_labels)
    training_pred_part3 = np.array(model.predict(features_fitted_part3))
    test_pred_part3 = np.array(model.predict(test_features_fitted_part3))

#     print(c)
#     print('Overall Accuracy for train  ', accuracy_score(training_labels, training_pred_part3))
#     print('Class Accuracy for train ', confusion_matrix(training_labels, training_pred_part3, normalize="true").diagonal())
#     print('Overall Accuracy : ', accuracy_score(test_labels, test_pred_part3))
#     print('Class Accuracy : ', confusion_matrix(test_labels, test_pred_part3, normalize="true").diagonal())
for c in np.linspace(5,10,100):
    svc = SVC(kernel='linear', class_weight='balanced', C=c)
    model = svc.fit(features_fitted_part3, training_labels)
    training_pred_part3 = np.array(model.predict(features_fitted_part3))
    test_pred_part3 = np.array(model.predict(test_features_fitted_part3))
    if(accuracy_score(test_labels, test_pred_part3)>0.458):
        print(c)
        print('Overall Accuracy for train  ', accuracy_score(training_labels, training_pred_part3))
        print('Class Accuracy for train ', confusion_matrix(training_labels, training_pred_part3, normalize="true").diagonal())
        print('Overall Accuracy : ', accuracy_score(test_labels, test_pred_part3))
        print('Class Accuracy : ', confusion_matrix(test_labels, test_pred_part3, normalize="true").diagonal())
c=5.1
C_values.append(c)
svc = SVC(kernel='linear', class_weight='balanced', C=c)
model = svc.fit(features_fitted_part3, training_labels)
training_pred_lin_part3 = np.array(model.predict(features_fitted_part3))
test_pred_lin_part3 = np.array(model.predict(test_features_fitted_part3))

5.0
Overall Accuracy for train   0.6344086021505376
Class Accuracy for train  [0.3        0.77272727 0.84210526]
Overall Accuracy :  0.4583333333333333
Class Accuracy :  [0.20833333 0.52631579 0.66666667]
5.05050505050505
Overall Accuracy for train   0.6397849462365591
Class Accuracy for train  [0.31666667 0.77272727 0.84210526]
Overall Accuracy :  0.4583333333333333
Class Accuracy :  [0.20833333 0.52631579 0.66666667]
5.101010101010101
Overall Accuracy for train   0.6344086021505376
Class Accuracy for train  [0.3        0.77272727 0.84210526]
Overall Accuracy :  0.4583333333333333
Class Accuracy :  [0.20833333 0.52631579 0.66666667]


## Found best C and gamma for RBF kernel

In [342]:
## Found best C for linear kernel
for c in C_values:
    for gamma in Gamma_values:
        svc = SVC(kernel='rbf', class_weight='balanced', C=c, gamma=gamma)
        model = svc.fit(features_fitted_part3, training_labels)
        training_pred_part3 = np.array(model.predict(features_fitted_part3))
        test_pred_part3 = np.array(model.predict(test_features_fitted_part3))
        if(accuracy_score(test_labels, test_pred_part3)>0.5):
            print(c)
            print(gamma)
            print('Overall Accuracy for train  ', accuracy_score(training_labels, training_pred_part3))
            print('Class Accuracy for train ', confusion_matrix(training_labels, training_pred_part3, normalize="true").diagonal())
            print('Overall Accuracy : ', accuracy_score(test_labels, test_pred_part3))
            print('Class Accuracy : ', confusion_matrix(test_labels, test_pred_part3, normalize="true").diagonal())

c=10            
gamma=0.4

svc = SVC(kernel='rbf', class_weight='balanced', C=c, gamma=gamma)
model = svc.fit(features_fitted_part3, training_labels)
training_pred_rbf_part3 = np.array(model.predict(features_fitted_part3))
test_pred_rbf_part3 = np.array(model.predict(test_features_fitted_part3))

10
0.4
Overall Accuracy for train   0.7903225806451613
Class Accuracy for train  [0.51666667 0.92045455 0.92105263]
Overall Accuracy :  0.5069444444444444
Class Accuracy :  [0.29166667 0.63157895 0.58974359]


In [343]:
np.savetxt('test_pred_lin_part3', test_pred_lin_part3, delimiter=',')
np.savetxt('test_pred_rbf_part3', test_pred_rbf_part3, delimiter=',')

In [348]:
training_pred_lin_part3 = np.array(training_pred_lin_part3)
training_pred_rbf_part3 = np.array(training_pred_rbf_part3)
test_pred_lin_part3 = np.array(test_pred_lin_part3)
test_pred_rbf_part3 = np.array(test_pred_rbf_part3)

for i in range(4):
    table_train_lin_2_3 = contingency_table(training_labels, training_pred_lin, training_pred_lin_part3, i)
    result_train_lin_2_3 = mcnemar(table_train_lin_2_3, exact=False, correction=True)
    print("Class \n",i)
    print("Result_Train\n",result_train_lin_2_3)
    
    table_test_lin_2_3 = contingency_table(test_labels, test_pred_lin, test_pred_lin_part3, i)
    result_test_lin_2_3 = mcnemar(table_test_lin_2_3, exact=False, correction=True)
    print("Result_Test\n",result_test_lin_2_3)

Class 
 0
Result_Train
 pvalue      0.4237107971667936
statistic   0.64
Result_Test
 pvalue      0.21129954733370696
statistic   1.5625
Class 
 1
Result_Train
 pvalue      0.2888443663464818
statistic   1.125
Result_Test
 pvalue      0.6170750774519739
statistic   0.25
Class 
 2
Result_Train
 pvalue      1.0
statistic   0.0
Result_Test
 pvalue      0.04550026389635857
statistic   4.0
Class 
 3
Result_Train
 pvalue      0.6170750774519739
statistic   0.25
Result_Test
 pvalue      1.0
statistic   0.0


In [347]:
for i in range(4):
    table_train_rbf_2_3 = contingency_table(training_labels, training_pred_rbf, training_pred_rbf_part3, i)
    result_train_rbf_2_3 = mcnemar(table_train_rbf_2_3, exact=False, correction=True)
    print("Class \n",i)
    print("Result_Train\n",result_train_rbf_2_3)
    
    table_test_rbf_2_3 = contingency_table(test_labels, test_pred_rbf, test_pred_rbf_part3, i)  
    result_test_rbf_2_3 = mcnemar(table_test_rbf_2_3, exact=False, correction=True)
    print("Result_Test\n",result_test_rbf_2_3)

Class 
 0
Result_Train
 pvalue      0.16142946236707922
statistic   1.9607843137254901
Result_Test
 pvalue      0.014688191945847787
statistic   5.953488372093023
Class 
 1
Result_Train
 pvalue      0.48623432138829725
statistic   0.48484848484848486
Result_Test
 pvalue      0.0005120045221899044
statistic   12.071428571428571
Class 
 2
Result_Train
 pvalue      0.7518296340458492
statistic   0.1
Result_Test
 pvalue      0.024448945310089332
statistic   5.0625
Class 
 3
Result_Train
 pvalue      0.01332832878081758
statistic   6.125
Result_Test
 pvalue      0.0008740872112984756
statistic   11.076923076923077


# Part IV

## Note that below part is take so long so go below and direclty upload the datas from features which I already get features data and store in cvs

In [380]:
## Note that below part is take so long so go below and direclty upload the datas from features which I already get features data and store in cvsTrain_features_part4 = []
acc_m_list = []
for img_file in glob.glob(test_dir + "*.jpg"):
    img_initial = cv.imread(img_file, 0)
    #keypoints
    sift = cv2.xfeatures2d.SIFT_create()
    keypoints_1, descriptors_1 = sift.detectAndCompute(img_initial,None)
    img = cv2.drawKeypoints(img_initial,keypoints_1,img_initial)
    acc_m = calculateAccumulatedCooccurrenceMatrix(patch, bin_number, d)
    acc_m_list.append(calculateCooccurrenceFeatures(acc_m))
    Train_features_part4.append(np.mean((np.array(acc_m_list)), 0))
    
Train_features_part4 = np.array(Train_features_part4)
np.savetxt('Train_features_part4.csv', Train_features_part4, delimiter=',')

## Note that below part is take so long so go below and direclty upload the datas from features which I already get features data and store in cvs

In [ ]:
## Note that below part is take so long so go below and direclty upload the datas from features which I already get features data and store in cvsTest_features_part4 = []
acc_m_list = []
for img_file in glob.glob(test_dir + "*.jpg"):
    img_initial = cv.imread(img_file, 0)
    #keypoints
    sift = cv2.xfeatures2d.SIFT_create()
    keypoints_1, descriptors_1 = sift.detectAndCompute(img_initial,None)
    img = cv2.drawKeypoints(img_initial,keypoints_1,img_initial)
    acc_m = calculateAccumulatedCooccurrenceMatrix(patch, bin_number, d)
    acc_m_list.append(calculateCooccurrenceFeatures(acc_m))
    Test_features_part4.append(np.mean((np.array(acc_m_list)), 0))
    
Test_features_part4 = np.array(Test_features_part4)
np.savetxt('Test_features_part4.csv', Test_features_part4, delimiter=',')

## Train Features

In [381]:
training_labels = np.loadtxt("dataset//training_labels.txt")
training_labels = training_labels.tolist()
training_labels = np.array(training_labels)## Note that below part is take so long so go below and direclty upload the datas from features which I already get features data and store in txt
# load from the csv file
train_features_part4 = np.loadtxt('train_features_part4.csv',delimiter=',')

## Test Features

In [382]:
test_labels = np.loadtxt("dataset//test_labels.txt")
test_labels = test_labels.tolist()
test_labels = np.array(test_labels)
# load from the csv file
test_features_part4 = np.loadtxt('test_features_part4.csv',delimiter=',')

## Found best C for linear kernel

In [385]:

features_fitted_part4 = scaler.fit_transform(train_features_part4)
test_features_fitted_part4 = scaler.fit_transform(test_features_part4)
features_fitted_part4 = np.array(features_fitted_part4)
test_features_fitted_part4 = np.array(test_features_fitted_part4)
for c in C_values:
    svc = SVC(kernel='linear', class_weight='balanced', C=c)
    model = svc.fit(features_fitted_part4, training_labels)
    training_pred_part4 = np.array(model.predict(features_fitted_part4))
    test_pred_part4 = np.array(model.predict(test_features_fitted_part4))

    print(c)
    print('Overall Accuracy for train  ', accuracy_score(training_labels, training_pred_part3))
    print('Class Accuracy for train ', confusion_matrix(training_labels, training_pred_part3, normalize="true").diagonal())
    print('Overall Accuracy : ', accuracy_score(test_labels, test_pred_part3))
    print('Class Accuracy : ', confusion_matrix(test_labels, test_pred_part3, normalize="true").diagonal())
for c in np.linspace(5,10,100):
    svc = SVC(kernel='linear', class_weight='balanced', C=c)
    model = svc.fit(features_fitted_part4, training_labels)
    training_pred_part4 = np.array(model.predict(features_fitted_part4))
    test_pred_part4 = np.array(model.predict(test_features_fitted_part4))
    if(accuracy_score(test_labels, test_pred_part4)>0.458):
        print(c)
        print('Overall Accuracy for train  ', accuracy_score(training_labels, training_pred_part4))
        print('Class Accuracy for train ', confusion_matrix(training_labels, training_pred_part4, normalize="true").diagonal())
        print('Overall Accuracy : ', accuracy_score(test_labels, test_pred_part4))
        print('Class Accuracy : ', confusion_matrix(test_labels, test_pred_part4, normalize="true").diagonal())
c=5.1
C_values.append(c)
svc = SVC(kernel='linear', class_weight='balanced', C=c)
model = svc.fit(features_fitted_part4, training_labels)
training_pred_lin_part4 = np.array(model.predict(features_fitted_part4))
test_pred_lin_part4 = np.array(model.predict(test_features_fitted_part4))

0.1
Overall Accuracy for train   0.9946236559139785
Class Accuracy for train  [0.98333333 1.         1.        ]
Overall Accuracy :  0.4444444444444444
Class Accuracy :  [0.1875     0.87719298 0.12820513]
1
Overall Accuracy for train   0.9946236559139785
Class Accuracy for train  [0.98333333 1.         1.        ]
Overall Accuracy :  0.4444444444444444
Class Accuracy :  [0.1875     0.87719298 0.12820513]
5
Overall Accuracy for train   0.9946236559139785
Class Accuracy for train  [0.98333333 1.         1.        ]
Overall Accuracy :  0.4444444444444444
Class Accuracy :  [0.1875     0.87719298 0.12820513]
10
Overall Accuracy for train   0.9946236559139785
Class Accuracy for train  [0.98333333 1.         1.        ]
Overall Accuracy :  0.4444444444444444
Class Accuracy :  [0.1875     0.87719298 0.12820513]
50
Overall Accuracy for train   0.9946236559139785
Class Accuracy for train  [0.98333333 1.         1.        ]
Overall Accuracy :  0.4444444444444444
Class Accuracy :  [0.1875     0.87

## Found best C and gamma for RBF kernel

In [403]:
## Found best C for linear kernel
for c in C_values:
    for gamma in Gamma_values:
        svc = SVC(kernel='rbf', class_weight='balanced', C=c, gamma=gamma)
        model = svc.fit(features_fitted_part4, training_labels)
        training_pred_part4 = np.array(model.predict(features_fitted_part4))
        test_pred_part4 = np.array(model.predict(test_features_fitted_part4))
        if(accuracy_score(test_labels, test_pred_part4)>0.56):
            print(c)
            print(gamma)
            print('Overall Accuracy for train  ', accuracy_score(training_labels, training_pred_part4))
            print('Class Accuracy for train ', confusion_matrix(training_labels, training_pred_part4, normalize="true").diagonal())
            print('Overall Accuracy : ', accuracy_score(test_labels, test_pred_part4))
            print('Class Accuracy : ', confusion_matrix(test_labels, test_pred_part4, normalize="true").diagonal())

c=5000           
gamma=0.3

svc = SVC(kernel='rbf', class_weight='balanced', C=c, gamma=gamma)
model = svc.fit(features_fitted_part4, training_labels)
training_pred_rbf_part4 = np.array(model.predict(features_fitted_part4))
test_pred_rbf_part4 = np.array(model.predict(test_features_fitted_part4))

0.1
0.1
Overall Accuracy for train   0.7903225806451613
Class Accuracy for train  [1.         0.98863636 0.        ]
Overall Accuracy :  0.6666666666666666
Class Accuracy :  [0.8125 1.     0.    ]
5000
0.30000000000000004
Overall Accuracy for train   1.0
Class Accuracy for train  [1. 1. 1.]
Overall Accuracy :  0.5625
Class Accuracy :  [0.72916667 0.80701754 0.        ]


In [401]:
## Found best C for linear kernel
for c in np.linspace(1000,10000,1000):
    for gamma in Gamma_values:
        svc = SVC(kernel='rbf', class_weight='balanced', C=c, gamma=gamma)
        model = svc.fit(features_fitted_part4, training_labels)
        training_pred_part4 = np.array(model.predict(features_fitted_part4))
        test_pred_part4 = np.array(model.predict(test_features_fitted_part4))
        if(accuracy_score(test_labels, test_pred_part4)>0.56):
            print(c)
            print(gamma)
            print('Overall Accuracy for train  ', accuracy_score(training_labels, training_pred_part4))
            print('Class Accuracy for train ', confusion_matrix(training_labels, training_pred_part4, normalize="true").diagonal())
            print('Overall Accuracy : ', accuracy_score(test_labels, test_pred_part4))
            print('Class Accuracy : ', confusion_matrix(test_labels, test_pred_part4, normalize="true").diagonal())

c=5000            
gamma=0.3

svc = SVC(kernel='rbf', class_weight='balanced', C=c, gamma=gamma)
model = svc.fit(features_fitted_part4, training_labels)
training_pred_rbf_part4 = np.array(model.predict(features_fitted_part4))
test_pred_rbf_part4 = np.array(model.predict(test_features_fitted_part4))

4720.720720720721
0.30000000000000004
Overall Accuracy for train   1.0
Class Accuracy for train  [1. 1. 1.]
Overall Accuracy :  0.5625
Class Accuracy :  [0.72916667 0.80701754 0.        ]
4729.72972972973
0.30000000000000004
Overall Accuracy for train   1.0
Class Accuracy for train  [1. 1. 1.]
Overall Accuracy :  0.5625
Class Accuracy :  [0.72916667 0.80701754 0.        ]
4738.738738738739
0.30000000000000004
Overall Accuracy for train   1.0
Class Accuracy for train  [1. 1. 1.]
Overall Accuracy :  0.5625
Class Accuracy :  [0.72916667 0.80701754 0.        ]
4747.747747747748
0.30000000000000004
Overall Accuracy for train   1.0
Class Accuracy for train  [1. 1. 1.]
Overall Accuracy :  0.5625
Class Accuracy :  [0.72916667 0.80701754 0.        ]
4756.756756756757
0.30000000000000004
Overall Accuracy for train   1.0
Class Accuracy for train  [1. 1. 1.]
Overall Accuracy :  0.5625
Class Accuracy :  [0.72916667 0.80701754 0.        ]
4765.7657657657655
0.30000000000000004
Overall Accuracy for t

In [409]:
training_pred_lin_part4 = np.array(training_pred_lin_part4)
training_pred_rbf_part4 = np.array(training_pred_rbf_part4)
test_pred_lin_part4 = np.array(test_pred_lin_part4)
test_pred_rbf_part4 = np.array(test_pred_rbf_part4)

for i in range(4):
    table_train_lin_2_3_4 = contingency_table(training_labels, training_pred_part3, training_pred_lin_part4, i)
    result_train_lin_2_3_4 = mcnemar(table_train_lin_2_3_4, exact=False, correction=True)
    print("Class \n",i)
    print("Result_Train\n",result_train_lin_2_3_4)
    
    table_test_lin_2_3_4 = contingency_table(test_labels, test_pred_lin_part3, test_pred_lin_part4, i)
    result_test_lin_2_3_4 = mcnemar(table_test_lin_2_3_4, exact=False, correction=True)
    print("Result_Test\n",result_test_lin_2_3_4)

Class 
 0
Result_Train
 pvalue      3.8596164369284525e-06
statistic   21.333333333333332
Result_Test
 pvalue      0.5897737906589077
statistic   0.29069767441860467
Class 
 1
Result_Train
 pvalue      1.0
statistic   0.0
Result_Test
 pvalue      1.6286575766479503e-05
statistic   18.580645161290324
Class 
 2
Result_Train
 pvalue      9.443043861123425e-07
statistic   24.03846153846154
Result_Test
 pvalue      0.2811980995641761
statistic   1.1612903225806452
Class 
 3
Result_Train
 pvalue      0.0
statistic   inf
Result_Test
 pvalue      2.6679545936247906e-06
statistic   22.041666666666668


In [407]:
for i in range(4):
    table_train_rbf_2_3_4 = contingency_table(training_labels, training_pred_rbf_part3, training_pred_rbf_part4, i)
    result_train_rbf_2_3_4 = mcnemar(table_train_rbf_2_3_4, exact=False, correction=True)
    print("Class \n",i)
    print("Result_Train\n",result_train_rbf_2_3_4)
    
    table_test_rbf_2_3_4 = contingency_table(test_labels, test_pred_rbf_part3, test_pred_rbf_part4, i)  
    result_test_rbf_2_3_4 = mcnemar(table_test_rbf_2_3_4, exact=False, correction=True)
    print("Result_Test\n",result_test_rbf_2_3_4)

Class 
 0
Result_Train
 pvalue      1.1658598093780509e-09
statistic   37.02564102564103
Result_Test
 pvalue      0.40939548620991884
statistic   0.6805555555555556
Class 
 1
Result_Train
 pvalue      1.9985815081381854e-07
statistic   27.03448275862069
Result_Test
 pvalue      0.00011860013866224722
statistic   14.814814814814815
Class 
 2
Result_Train
 pvalue      0.02334220201289086
statistic   5.142857142857143
Result_Test
 pvalue      0.055008833629265896
statistic   3.6818181818181817
Class 
 3
Result_Train
 pvalue      0.24821307898992026
statistic   1.3333333333333333
Result_Test
 pvalue      4.489784498638631e-06
statistic   21.043478260869566
